In [1]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import base64
import io
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import statsmodels.api as sm
import numpy as np
from scipy.stats import ttest_1samp, shapiro, levene, f_oneway, ttest_ind, chi2_contingency, mannwhitneyu, ks_2samp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import base64
import dash_bootstrap_components as dbc
# Étape 1 : Générer les données synthétiques, entraîner le modèle et le scaler
# Données synthétiques
data = {
    "Patient_ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "age": [65, 72, 68, 80, 70, 75, 82, 66, 78, 69],
    "mmse_score": [28, 24, 26, 20, 27, 22, 18, 29, 23, 25],
    "cdr": [0.0, 0.5, 0.0, 1.0, 0.0, 0.5, 1.5, 0.0, 1.0, 0.5],
    "education_level": [12, 10, 16, 8, 14, 11, 9, 15, 12, 13],
    "family_history": [0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
    "Sexe": ["M", "F", "F", "M", "M", "F", "F", "M", "F", "M"],
    "Résultat": [0, 1, 0, 1, 0, 1, 1, 0, 1, 0]
}

# Création du DataFrame
df_synthetic = pd.DataFrame(data)

# Sélection des caractéristiques et de la cible
features = ['age', 'mmse_score', 'cdr', 'education_level', 'family_history']
X = df_synthetic[features]
y = df_synthetic['Résultat']

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des données avec StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraînement du modèle Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Sauvegarde du modèle et du scaler
joblib.dump(model, 'alzheimer_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Modèle et scaler sauvegardés avec succès sous 'alzheimer_model.pkl' et 'scaler.pkl'.")

# Étape 2 : Code du tableau de bord (inchangé sauf pour le callback de prédiction)
# Initialisation de l'application
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG], suppress_callback_exceptions=True)
df = pd.DataFrame()

# Charger le modèle ML et le scaler pour la prédiction
try:
    model = joblib.load('alzheimer_model.pkl')
    scaler = joblib.load('scaler.pkl')
    print("Modèle et scaler chargés avec succès.")
except FileNotFoundError:
    print("Erreur : Les fichiers 'alzheimer_model.pkl' et 'scaler.pkl' doivent être dans le même répertoire.")
    raise Exception("Impossible de continuer sans le modèle et le scaler.")

# Ajout de CSS personnalisé pour les dropdowns
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})

# CSS personnalisé pour les dropdowns
custom_css = """
.dropdown-custom .Select-control {
    background-color: #333333 !important;
    color: black !important;
    border: 2px solid purple !important;
    border-radius: 8px !important;
}
.dropdown-custom .Select-menu-outer {
    background-color: #333333 !important;
    color: black !important;
    border: 2px solid purple !important;
}
.dropdown-custom .Select-option {
    background-color: #333333 !important;
    color: black !important;
}
.dropdown-custom .Select-option.is-focused {
    background-color: #4B0082 !important;
    color: white !important;
}
"""

# CSS pour l'animation du titre
title_animation_css = """
@keyframes fadeInSlideUpGlow {
    0% {
        opacity: 0;
        transform: translateY(20px);
        text-shadow: 2px 2px 4px rgba(200, 150, 0, 0.2);
    }
    50% {
        opacity: 0.5;
        transform: translateY(10px);
        text-shadow: 2px 2px 8px rgba(200, 150, 0, 0.5);
    }
    100% {
        opacity: 1;
        transform: translateY(0);
        text-shadow: 2px 2px 4px rgba(200, 150, 0, 0.5);
    }
}

.animated-title {
    animation: fadeInSlideUpGlow 1.5s ease-in-out forwards;
}
"""

# Combiner les CSS
combined_css = custom_css + title_animation_css
app.css.append_css({"external_url": f"data:text/css;base64,{base64.b64encode(combined_css.encode()).decode()}"})

def render_table(df, num_rows):
    """ Génère un tableau Dash avec des champs éditables. """
    if df.empty:
        return html.P("Aucun fichier chargé.", style={"color": "gray"})
    
    table_header = [html.Tr([html.Th(col) for col in df.columns] + [html.Th("Actions")])]
    table_body = [
        html.Tr([
            html.Td(dcc.Input(value=df.iloc[i][col], id={'type': 'editable-cell', 'index': i, 'column': col}, style={'width': '100%'}))
            for col in df.columns
        ] + [html.Td(html.Button("Supprimer", id={'type': 'delete-row', 'index': i}, n_clicks=0))])
        for i in range(min(num_rows, len(df)))
    ]
    return dbc.Table(table_header + table_body, striped=True, bordered=True, hover=True, style={"color": "white"})

# Interface d'imputation
def render_imputation_interface(column_options):
    return [
        html.H4("Imputation des valeurs manquantes", style={"color": "lightblue"}),
        html.Label("Sélectionnez une colonne à imputer", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='impute-column',
            options=column_options,
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Sélectionnez une méthode d'imputation", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='impute-method',
            options=[],
            placeholder="Sélectionnez une méthode",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Button("Appliquer l'imputation", id="apply-impute-btn", n_clicks=0, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        html.Button("Résultats après imputation", id="show-results-btn", n_clicks=0, disabled=True, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        html.Button("Valider les données", id="validate-data-btn", n_clicks=0, disabled=True, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        html.Button("Télécharger les données", id="download-data-btn", n_clicks=0, disabled=True, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        dcc.Download(id="download-data"),
        html.P(id='impute-message', style={"color": "red"}),
        html.P(id='download-message', style={"color": "red"}),
        html.Div(id='impute-results', style={"marginTop": "20px"})
    ]

# Interface pour la visualisation des distributions
def render_distribution_interface(numeric_columns):
    return [
        html.H4("Visualisation des Distributions", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour la visualisation", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='dist-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Type de visualisation", style={"color": "lightgray"}),
        dcc.RadioItems(
            id='dist-type',
            options=[
                {'label': 'Densité', 'value': 'density'},
                {'label': 'Q-Q Plot', 'value': 'qqplot'}
            ],
            value='density',
            labelStyle={'display': 'block', 'color': 'lightgray'}
        ),
        dcc.Graph(id='dist-graph')
    ]

# Interface pour les visualisations statistiques
def render_statistics_interface(numeric_columns, all_columns):
    return [
        html.H4("Histogramme", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour l'histogramme", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='hist-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Nombre de barres :", style={"color": "lightgray"}),
        dcc.Slider(
            id='hist-bins',
            min=1,
            max=50,
            step=1,
            value=30,
            marks={i: str(i) for i in range(1, 51, 5)},
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        dcc.Graph(id='hist-graph'),

        html.H4("Diagramme en boîte", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour le diagramme en boîte", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='box-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Largeur du boîte :", style={"color": "lightgray"}),
        dcc.Slider(
            id='box-width',
            min=0.1,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(1, 11)},
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        dcc.Graph(id='box-graph'),

        html.H4("Histogramme groupé", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour grouper l'histogramme", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='group-hist-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Sélectionner la colonne pour les valeurs de l'histogramme", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='group-hist-value-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        dcc.Graph(id='group-hist-graph'),

        html.H4("Diagramme en boîte groupé", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour grouper le boxplot", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='group-box-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Sélectionner la colonne pour les valeurs du boxplot", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='group-box-value-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        dcc.Graph(id='group-box-graph'),

        html.H4("Diagramme à secteurs", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour le diagramme à secteurs", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='pie-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        dcc.Graph(id='pie-graph'),

        html.H4("Graphique croisé", style={"color": "lightblue"}),
        html.Label("Sélectionner la colonne pour l'axe des X", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='stacked-x-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Sélectionner la colonne pour l'axe des Y", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='stacked-y-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Label("Sélectionner la colonne pour la couleur", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='stacked-color-column',
            options=[{'label': col, 'value': col} for col in all_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        dcc.Graph(id='stacked-graph')
    ]

# Interface pour la visualisation des corrélations
def render_correlation_interface(numeric_columns):
    if not numeric_columns:
        return [html.P("Aucune colonne numérique disponible pour la visualisation des corrélations.", style={"color": "gray"})]
    
    return [
        html.H4("Visualisation des Corrélations", style={"color": "lightblue"}),
        html.Label("Méthode de visualisation de la corrélation", style={"color": "lightgray"}),
        dcc.RadioItems(
            id='corr-method',
            options=[
                {'label': 'Nombre', 'value': 'number'},
                {'label': 'Cercle', 'value': 'circle'}
            ],
            value='circle',
            labelStyle={'display': 'inline-block', 'marginRight': '20px', 'color': 'lightgray'}
        ),
        dcc.Graph(id='corr-graph')
    ]

# Interface pour la régression linéaire
def render_regression_interface(numeric_columns):
    if not numeric_columns:
        return [html.P("Aucune colonne numérique disponible pour la régression linéaire.", style={"color": "gray"})]
    
    return [
        html.H4("Régression Linéaire", style={"color": "lightblue", "fontWeight": "bold", "textShadow": "1px 1px 2px rgba(255, 215, 0, 0.3)"}),
        html.Label("Variable Explicative (X)", style={"color": "lightgray", "fontWeight": "bold"}),
        dcc.Dropdown(
            id='regression-x-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "15px", 
                "backgroundColor": "#333333",
                "color": "black",
                "border": "2px solid purple",
                "borderRadius": "8px"
            },
            className="dropdown-custom",
        ),
        html.Label("Variable à Expliquer (Y)", style={"color": "lightgray", "fontWeight": "bold"}),
        dcc.Dropdown(
            id='regression-y-column',
            options=[{'label': col, 'value': col} for col in numeric_columns],
            placeholder="Sélectionnez une colonne",
            style={
                "width": "50%", 
                "marginBottom": "15px", 
                "backgroundColor": "#333333",
                "color": "black",
                "border": "2px solid purple",
                "borderRadius": "8px"
            },
            className="dropdown-custom",
        ),
        html.Button("Exécuter la régression linéaire", id="run-regression-btn", n_clicks=0, style={
            "margin": "10px", 
            "fontWeight": "bold", 
            "backgroundColor": "#4B0082",
            "color": "white",
            "border": "2px solid purple", 
            "borderRadius": "8px",
            "padding": "10px 20px",
            "textTransform": "uppercase"
        }),
        html.Button("Télécharger les résultats", id="download-regression-btn", n_clicks=0, disabled=True, style={
            "margin": "10px", 
            "fontWeight": "bold", 
            "backgroundColor": "#4B0082",
            "color": "white",
            "border": "2px solid purple", 
            "borderRadius": "8px",
            "padding": "10px 20px",
            "textTransform": "uppercase"
        }),
        dcc.Download(id="download-regression"),
        html.P(id='regression-message', style={"color": "red", "fontWeight": "bold"}),
        html.Div(id='regression-results')
    ]

# Interface pour les tests statistiques (ajustée pour les 4 tests demandés)
def render_tests_interface(all_columns):
    if not all_columns:
        return [html.P("Aucune colonne disponible pour les tests.", style={"color": "gray"})]
    
    return [
        html.H4("Tests Statistiques", style={"color": "lightblue"}),
        html.Label("Sélectionner le type de test", style={"color": "lightgray"}),
        dcc.Dropdown(
            id='test-type',
            options=[
                {'label': 't-test', 'value': 't-test'},
                {'label': 'Test de normalité (Shapiro-Wilk)', 'value': 'normality'},
                {'label': 'Test Chi-squared', 'value': 'chi-squared'},
                {'label': 'ANOVA', 'value': 'anova'}
            ],
            placeholder="Sélectionnez un test",
            style={
                "width": "50%", 
                "marginBottom": "10px", 
                "backgroundColor": "white", 
                "color": "black"
            }
        ),
        html.Div(id='test-options-container', style={"marginTop": "10px"}),
        html.Button("Exécuter le test", id="run-test-btn", n_clicks=0, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        html.Button("Télécharger les résultats", id="download-test-btn", n_clicks=0, disabled=True, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        }),
        dcc.Download(id="download-test"),
        html.P(id='test-message', style={"color": "red"}),
        html.Div(id='test-results', style={"marginTop": "20px"}),
        # Ajout des graphiques pour chaque test
        dcc.Graph(id='t-test-graph', style={'display': 'none'}),
        dcc.Graph(id='normality-graph', style={'display': 'none'}),
        dcc.Graph(id='chi-squared-graph', style={'display': 'none'}),
        dcc.Graph(id='anova-graph', style={'display': 'none'}),
        # Placeholders ajustés pour les composants
        dcc.Dropdown(id='test-column', options=[], value=None, style={'display': 'none'}),
        dcc.Dropdown(id='group-column', options=[], value=None, style={'display': 'none'}),
        dcc.Dropdown(id='value-column', options=[], value=None, style={'display': 'none'}),
        dcc.Dropdown(id='chi2-column1', options=[], value=None, style={'display': 'none'}),
        dcc.Dropdown(id='chi2-column2', options=[], value=None, style={'display': 'none'})
    ]
# Interface pour la prédiction de maladie
def render_prediction_interface():
    return [
        html.H4("Prédiction de la Maladie d'Alzheimer (Présente ou Absente)", style={"color": "lightblue"}),
        html.Label("Charger les données du patient (fichier CSV)", style={"color": "lightgray"}),
        dcc.Upload(
            id='upload-prediction-data',
            children=html.Button('Charger les données', style={
                "backgroundColor": "#444",
                "color": "white",
                "border": "1px solid purple",
                "marginBottom": "10px"
            }),
            multiple=False
        ),
        html.Button('Prédire', id='predict-btn', n_clicks=0, style={
            "backgroundColor": "#444",
            "color": "white",
            "border": "1px solid purple",
            "margin": "10px"
        }),
        html.Hr(),
        html.H5("Prédiction manuelle pour un patient", style={"color": "lightblue"}),
        html.Div([
            html.Label("Âge", style={"color": "lightgray"}),
            dcc.Input(id='input-age', type='number', value=70, style={"marginBottom": "10px", "width": "30%", "backgroundColor": "gray"}),
            html.Br(),
            html.Label("Score MMSE (0-30)", style={"color": "lightgray"}),
            dcc.Input(id='input-mmse', type='number', value=25, min=0, max=30, style={"marginBottom": "10px", "width": "30%", "backgroundColor": "gray"}),
            html.Br(),
            html.Label("CDR (0-3)", style={"color": "lightgray"}),
            dcc.Input(id='input-cdr', type='number', value=0.5, step=0.5, min=0, max=3, style={"marginBottom": "10px", "width": "30%", "backgroundColor": "gray"}),
            html.Br(),
            html.Label("Niveau d'éducation (années)", style={"color": "lightgray"}),
            dcc.Input(id='input-education', type='number', value=12, min=0, style={"marginBottom": "10px", "width": "30%", "backgroundColor": "gray"}),
            html.Br(),
            html.Label("Antécédents familiaux (0=Non, 1=Oui)", style={"color": "lightgray"}),
            dcc.Input(id='input-family', type='number', value=1, min=0, max=1, style={"marginBottom": "10px", "width": "30%", "backgroundColor": "gray"}),
            html.Br(),
            html.Button('Prédire manuellement', id='predict-manual-btn', n_clicks=0, style={
                "backgroundColor": "#444",
                "color": "white",
                "border": "1px solid purple",
                "margin": "10px"
            }),
        ]),
        html.Div(id='prediction-results', style={"color": "lightgray", "marginTop": "20px"})
    ]
# Après les imports ou avant app.layout
faq_data = {
    "demarrage": {
        "Comment utiliser cette application ?": "Pour utiliser cette application, commencez par uploader un fichier CSV via le bouton 'Charger votre fichier' dans la barre latérale. Une fois chargé, utilisez les onglets comme 'Imputation' pour nettoyer les données, 'Statistiques' pour visualiser, ou 'Prédiction' pour analyser avec un modèle d'apprentissage automatique. Assurez-vous que vos données incluent des colonnes comme 'age', 'mmse_score', etc., avec des en-têtes clairs.",
        "Quels types de fichiers puis-je uploader ?": "Vous pouvez uploader des fichiers CSV ou Excel. Les colonnes doivent contenir des données tabulaires avec des en-têtes (ex. : 'age', 'mmse_score'). Évitez les formats non structurés comme TXT.",
        "Quelle est la taille maximale d’un fichier ?": "La taille maximale est de 10 Mo pour un chargement fluide. Pour des fichiers plus grands, divisez-les en plusieurs parties ou contactez le support pour une assistance."
    },
    "nettoyage": {
        "Comment gérer les valeurs manquantes ?": "Dans l'onglet 'Imputation', sélectionnez une colonne (ex. : 'mmse_score') et choisissez une méthode (moyenne, médiane, mode) via le dropdown. Cliquez sur 'Appliquer l'imputation' pour traiter les données.",
        "Comment supprimer les doublons ?": "Dans l'onglet 'Imputation', cochez l'option 'Supprimer doublons' si disponible, ou utilisez une fonction personnalisée via un script Python si nécessaire.",
        "Comment gérer les valeurs aberrantes ?": "Utilisez l'onglet 'Statistiques' pour identifier les outliers avec les boxplots, puis ajustez-les manuellement dans 'Imputation' en filtrant les valeurs extrêmes."
    },
    "visualisation": {
        "Quels types de visualisations puis-je créer ?": "Dans l'onglet 'Statistiques', créez des histogrammes, boxplots ou graphiques groupés en sélectionnant une colonne (ex. : 'age') et en choisissant le type via les dropdowns.",
        "Comment visualiser les corrélations ?": "Accédez à l'onglet 'Corrélations', sélectionnez une méthode ('Nombre' ou 'Cercle'), et observez les relations entre colonnes numériques comme 'age' et 'mmse_score'."
    },
    "analyse": {
        "Comment effectuer des tests statistiques ?": "Dans l'onglet 'Tests', choisissez un test (ex. : t-test, ANOVA), sélectionnez les colonnes (ex. : 'group-column' pour regrouper), et cliquez sur 'Exécuter le test' pour obtenir les résultats.",
        "Comment faire une analyse de régression ?": "Dans l'onglet 'Régression linéaire', sélectionnez deux colonnes numériques (ex. : 'age' comme X, 'mmse_score' comme Y), puis cliquez sur 'Exécuter la régression linéaire' pour voir la droite et les métriques.",
        "Comment faire des prédictions ?": "Dans l'onglet 'Prédiction', chargez un fichier CSV ou entrez manuellement des valeurs (age, mmse_score, etc.), puis cliquez sur 'Prédire' pour obtenir un diagnostic d'Alzheimer avec une jauge de risque."
    },
    "prediction": {
        "Comment utiliser l'onglet Prédiction ?": "Dans l'onglet 'Prédiction', entrez manuellement des valeurs pour les colonnes comme 'age', 'mmse_score', et 'group-column', ou chargez un fichier CSV avec ces données. Cliquez sur 'Prédire' pour obtenir une probabilité de diagnostic d'Alzheimer sous forme de jauge de risque.",
        "Quelles colonnes sont nécessaires pour une prédiction ?": "Vous devez fournir au moins 'age' (numérique), 'mmse_score' (numérique), et 'group-column' (catégorique). Assurez-vous que ces colonnes existent dans votre fichier ou entrez-les manuellement.",
        "Comment interpréter les résultats de prédiction ?": "Le résultat est affiché sous forme de jauge de risque (0 à 100 %). Une valeur élevée (ex. : > 70 %) indique un risque accru d'Alzheimer. Consultez un professionnel de santé pour une interprétation approfondie."
    },
    "depanne": {
        "Que faire si l’application plante ?": "Si l'application plante, vérifiez que votre fichier CSV est valide et respecte la limite de 10 Mo. Rechargez la page ou consultez l'onglet 'Résumé' pour des erreurs spécifiques.",
        "Pourquoi la FAQ ne s’affiche pas toujours ?": "Assurez-vous de cliquer sur le bouton 'FAQ' dans la barre latérale. Si le problème persiste, vérifiez votre connexion ou redémarrez l'application."
    }
}
app.layout = html.Div([
    # Conteneur pour la page d'accueil
    html.Div(
        id='welcome-container',
        style={
            'display': 'block',
            'background': 'linear-gradient(135deg, #000035, #00008B)',
            'minHeight': '100vh',
            'display': 'flex',
            'flexDirection': 'column',
            'justifyContent': 'center',
            'alignItems': 'center',
            'padding': '50px',
            'boxSizing': 'border-box',
        },
        children=[
            html.H1([
                "Bienvenue sur votre",
                html.Br(),
                "tableau de bord d'Analyse de Données"
            ],
            style={
                "color": "#B784A7",
                "textAlign": "center",
                "fontFamily": "Poppins, sans-serif",
                "fontSize": "60px",
                "fontWeight": "bold",
                "textShadow": "2px 2px 4px rgba(250, 200, 0, 0.5)",
                "marginBottom": "20px",
                "padding": "10px",
                "borderRadius": "10px",
                "backgroundColor": "#000035"
            }),
            html.P([
                "Plongez dans une expérience d'analyse de données inégalée !",
                html.Br(),
                "📂 Importez facilement vos fichiers CSV ou Excel en un seul clic pour commencer .",
                html.Br(),
                "📊Analysez vos données en profondeur grâce à des visualisations interactives et des outils puissants .",
                html.Br(),
                "Gérez les données manquantes en utilisant des méthodes statistiques comme la moyenne, la médiane, ou même des algorithmes avancés .",
                html.Br(),
                "🔍Effectuez des tests statistiques pour découvrir des relations significatives dans vos données .",
                html.Br(),
                "Obtenez des prédictions avancées avec des modèles d'apprentissage automatique en quelques étapes simples ."
            ],
            style={
                'textAlign': 'center',
                'color': '#FBE4D8',
                'fontWeight': 'bold',
                'textShadow': '2px 2px 4px rgba(150, 100, 0, 0.5)',
                'fontFamily': 'Roboto, sans-serif',
                'fontSize': '25px',
                'lineHeight': '1.6',
                'marginTop': '20px',
                'marginBottom': '40px'
            }),
            # Bouton pour passer à l'interface principale
            html.Div(
                html.Button(
                    "Commencer l'Exploration",
                    id="start-btn",
                    style={
                        'backgroundColor': '#522B5B',
                        'color': '#FBE4D8',
                        'border': '2px solid #884DA7',
                        'borderRadius': '12px',
                        'fontWeight': 'bold',
                        'fontFamily': 'Roboto, sans-serif',
                        'fontSize': '20px',
                        'padding': '15px 40px',
                        'marginTop': '20px',
                        'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.3)'
                    }
                ),
                style={'textAlign': 'center'}
            )
        ]
    ),
    # Conteneur principal (contient ton interface existante)
    html.Div(
        id='main-container',
        style={'display': 'none'},  # Caché par défaut
        children=[
            html.H1("Dashboard: Analyse de Données", className="animated-title", style={
                "color": "#FBE4D8",
                "textAlign": "center",
                "fontFamily": "Poppins, sans-serif",
                "fontSize": "50px",
                "fontWeight": "bold",
                "textShadow": "2px 2px 4px rgba(250, 200, 0, 0.5)",
                "marginBottom": "20px",
                "padding": "10px",
                "borderRadius": "10px",
                "backgroundColor": "#000035",
            }),

            dcc.Store(id='temp-data-store', data=None),
            dcc.Store(id='display-state', data='imputation'),
            dcc.Store(id='regression-results-store', data=None),
            dcc.Store(id='test-results-store', data=None),

            html.Div(
                className="sidebar",
                children=[
                    html.H2("Menu", style={"textAlign": "center", "color": "#FAF0E6", "font-weight": "bold", "textShadow": "2px 2px 4px rgba(150, 100, 0, 0.5)"}),
                    html.P("Chargement et Analyse des Données", style={"color": "#FBE4D8", "textAlign": "center", "fontWeight": "bold"}),
                    dcc.Upload(
                        id="upload-data",
                        children=[
                            html.Button("Retour à l'Accueil", id="back-to-welcome-btn", style={
                                "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                                "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                                "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                            }),
                            html.Button("Charger votre fichier", style={
                                "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                                "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                                "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                            }),
                        ],
                        multiple=False
                    ),
                    html.P("Combien de lignes voulez-vous afficher ?", style={"color": "#FBE4D8"}),
                    dcc.Input(id='num-rows', type='number', value=5, min=1, step=1, style={"margin": "10px", "width": "100%"}),
                    html.Button("Ajouter une ligne", id="add-row-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Résumé des données", id="summary-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Imputation", id="impute-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Distribution", id="dist-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Statistiques", id="stats-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Corrélations", id="corr-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Régression linéaire", id="regression-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Tests", id="tests-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("Prédiction", id="predict-section-btn", n_clicks=0, style={
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.Button("FAQ", id="faq-btn", n_clicks=0, style={  # Nouveau bouton FAQ
                        "margin": "10px", "fontWeight": "bold", "backgroundColor": "#522B5B",
                        "width": "100%", "color": "#FBE4D8", "border": "2px solid #884DA7",
                        "borderRadius": "8px", "font-family": "Roboto, sans-serif"
                    }),
                    html.P("Fichier chargé: Aucun", id="file-name", style={"color": "lightgray"})
                ],
                style={"width": "20%", "position": "fixed", "height": "100vh", "backgroundColor": "#000035", "padding": "20px", "overflowY": "auto"}
            ),

            html.Div(id='output-content', style={'marginLeft': '22%', 'padding': '20px', 'color': 'lightgray','minHeight': '100vh',
    'overflowY': 'auto',}, children=[
                html.Div(id='summary-table'),
                html.Div(id='search-container', style={'display': 'none'}),
                html.Div(id='impute-container', style={'display': 'none'}),
                html.Div(id='dist-container', style={'display': 'none'}),
                html.Div(id='stats-container', style={'display': 'none'}),
                html.Div(id='corr-container', style={'display': 'none'}),
                html.Div(id='regression-container', style={'display': 'none'}),
                html.Div(id='tests-container', style={'display': 'none'}),
                html.Div(id='predict-container', style={'display': 'none'}),
                html.Div(id='faq-container', style={'display': 'none'}, children=[  # Nouvelle section FAQ
                    html.H4("Questions Fréquentes", style={"color": "#FBE4D8"}),
                    dcc.Tabs(id="faq-tabs", value="demarrage", children=[
                        dcc.Tab(label="Démarrage", value="demarrage"),
                        dcc.Tab(label="Nettoyage de Données", value="nettoyage"),
                        dcc.Tab(label="Visualisation", value="visualisation"),
                        dcc.Tab(label="Analyse Statistique", value="analyse"),
                        dcc.Tab(label="Export & Rapports", value="export"),
                        dcc.Tab(label="Dépannage", value="depanne"),
                    ]),
                    html.Div(id="faq-content", style={"color": "white", "padding": "10px", "backgroundColor": "#1a2a44"}),
                ])
            ]),
        ]
    ),
    # Store pour gérer l'état de la page (accueil ou interface principale)
    dcc.Store(id='page-state', data='welcome')
], style={
    "backgroundColor": "#000035",
    "minHeight": "100vh",
})
@app.callback(
    [Output('output-content', 'children'),
     Output('file-name', 'children'),
     Output('temp-data-store', 'data'),
     Output('display-state', 'data')],
    [Input('upload-data', 'contents'),
     Input('num-rows', 'value'),
     Input('summary-btn', 'n_clicks'),
     Input('add-row-btn', 'n_clicks'),
     Input('impute-btn', 'n_clicks'),
     Input('dist-btn', 'n_clicks'),
     Input('stats-btn', 'n_clicks'),
     Input('corr-btn', 'n_clicks'),
     Input('regression-btn', 'n_clicks'),
     Input('tests-btn', 'n_clicks'),
     Input('predict-section-btn', 'n_clicks'),
     Input('faq-btn', 'n_clicks')],  # Ajout du bouton FAQ
    [State('upload-data', 'filename'),
     State('output-content', 'children'),
     State('temp-data-store', 'data'),
     State('page-state', 'data')]
)
def update_dashboard(contents, num_rows, n_summary, add_row_clicks, impute_clicks, dist_clicks, stats_clicks, corr_clicks, regression_clicks, tests_clicks, predict_clicks, faq_clicks, filename, current_output, temp_data, page_state):
    # Vérifier si nous sommes sur la page principale
    if page_state != 'main':
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update

    global df
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None

    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()

    if trigger_id == "upload-data" and contents:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        try:
            if filename.endswith('.csv'):
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            elif filename.endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(decoded))
            else:
                return html.P("Format de fichier non supporté. Utilisez CSV ou XLSX.", style={"color": "red"}), f"Fichier chargé: {filename}", None, 'imputation'
            
            df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
            df.replace(['nan', 'NaN', 'NA', 'missing', ''], pd.NA, inplace=True)
            df = df.where(df.notna(), pd.NA)

            def determine_column_type(col):
                try:
                    converted = pd.to_numeric(df[col], errors='coerce')
                    if converted.isna().mean() < 0.5:
                        return "Numérique"
                    else:
                        return "Qualitatif"
                except:
                    return "Qualitatif"

            column_types = {col: determine_column_type(col) for col in df.columns}
            
            for col, col_type in column_types.items():
                if col_type == "Numérique":
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                    def is_integer_value(x):
                        if pd.isna(x):
                            return True
                        if isinstance(x, int):
                            return True
                        if isinstance(x, float):
                            return x.is_integer()
                        return False

                    if df[col].dropna().apply(is_integer_value).all():
                        df[col] = df[col].astype('Int64')
                    else:
                        df[col] = df[col].astype('float64')

            df_temp = df.copy() if not df.empty else pd.DataFrame()
        except Exception as e:
            return html.P(f"Erreur lors du chargement du fichier : {str(e)}", style={"color": "red"}), f"Fichier chargé: {filename}", None, 'imputation'

    if trigger_id == "add-row-btn" and add_row_clicks > 0:
        if df.empty:
            return html.P("Veuillez charger un fichier avant d'ajouter une ligne.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        new_row = pd.DataFrame({col: [pd.NA] for col in df.columns}, index=[len(df)])
        df = pd.concat([df, new_row], ignore_index=True)
        df_temp = df.copy() if not df.empty else pd.DataFrame()

    if trigger_id == "impute-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        column_options = [{'label': col, 'value': col} for col in df.columns]
        
        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "block"}, children=render_imputation_interface(column_options)),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "dist-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        numeric_columns = [col for col in df.columns if pd.to_numeric(df[col], errors='coerce').notna().mean() > 0.5]
        
        if not numeric_columns:
            return html.P("Aucune colonne numérique disponible pour la visualisation.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "block"}, children=render_distribution_interface(numeric_columns)),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "stats-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        numeric_columns = [col for col in df.columns if pd.to_numeric(df[col], errors='coerce').notna().mean() > 0.5]
        all_columns = list(df.columns)
        
        if not all_columns:
            return html.P("Aucune colonne disponible pour la visualisation.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "block"}, children=render_statistics_interface(numeric_columns, all_columns)),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "corr-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        numeric_columns = [col for col in df.columns if pd.to_numeric(df[col], errors='coerce').notna().mean() > 0.5]
        
        if not numeric_columns:
            return html.P("Aucune colonne numérique disponible pour la visualisation des corrélations.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "block"}, children=render_correlation_interface(numeric_columns)),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "regression-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        numeric_columns = [col for col in df.columns if pd.to_numeric(df[col], errors='coerce').notna().mean() > 0.5]
        
        if not numeric_columns:
            return html.P("Aucune colonne numérique disponible pour la régression linéaire.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "block"}, children=render_regression_interface(numeric_columns)),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "tests-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        all_columns = list(df.columns)
        if not all_columns:
            return html.P("Aucune colonne disponible pour les tests.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "block"}, children=render_tests_interface(all_columns)),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "predict-section-btn":
        return html.Div([
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "block"}, children=render_prediction_interface()),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'

    if trigger_id == "faq-btn":  # Nouvelle logique pour la section FAQ
        return html.Div([
        html.Div(id='search-container', style={"display": "none"}),
        html.Div(id='impute-container', style={"display": "none"}),
        html.Div(id='dist-container', style={"display": "none"}),
        html.Div(id='stats-container', style={"display": "none"}),
        html.Div(id='corr-container', style={"display": "none"}),
        html.Div(id='regression-container', style={"display": "none"}),
        html.Div(id='tests-container', style={"display": "none"}),
        html.Div(id='predict-container', style={"display": "none"}),
        html.Div(id='faq-container', style={
            "display": "block",
            "backgroundColor": "#1a2a44",
            "color": "white",
            "padding": "30px",
            "borderRadius": "10px",
            "boxShadow": "0 4px 8px rgba(0, 0, 0, 0.2)",
            "margin": "20px"
        }, children=[
            html.H4("Questions Fréquentes", style={
                "color": "#FBE4D8",
                "marginBottom": "20px",
                "fontSize": "24px",
                "fontWeight": "bold"
            }),
            dcc.Tabs(id="faq-tabs", value="demarrage", children=[
                dcc.Tab(label="Démarrage", value="demarrage", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',  # Mauve pour le texte des onglets
                    'border': '1px solid #C71585',  # Bordure mauve
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',  # Fond mauve quand sélectionné
                    'color': '#1a2a44',  # Texte bleu nuit quand sélectionné
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
                dcc.Tab(label="Nettoyage de Données", value="nettoyage", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',
                    'border': '1px solid #C71585',
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',
                    'color': '#1a2a44',
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
                dcc.Tab(label="Visualisation", value="visualisation", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',
                    'border': '1px solid #C71585',
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',
                    'color': '#1a2a44',
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
                dcc.Tab(label="Analyse Statistique", value="analyse", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',
                    'border': '1px solid #C71585',
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',
                    'color': '#1a2a44',
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
                dcc.Tab(label="Prédiction", value="prediction", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',
                    'border': '1px solid #C71585',
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',
                    'color': '#1a2a44',
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
                dcc.Tab(label="Dépannage", value="depanne", style={
                    'backgroundColor': '#1a2a44',
                    'color': '#C71585',
                    'border': '1px solid #C71585',
                    'marginRight': '5px',
                    'padding': '10px',
                    'borderRadius': '5px'
                }, selected_style={
                    'backgroundColor': '#C71585',
                    'color': '#1a2a44',
                    'border': '1px solid #C71585',
                    'fontWeight': 'bold'
                }),
            ], style={'backgroundColor': '#1a2a44'}),
            html.Div(id="faq-content", style={
                "color": "white",
                "padding": "20px",
                "backgroundColor": "rgba(255, 255, 255, 0.05)",
                "borderRadius": "5px",
                "marginTop": "10px"
            })
        ]),
        html.Div(id='summary-table')
    ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'faq'
    if trigger_id == "summary-btn":
        if df.empty:
            return html.P("Veuillez charger un fichier.", style={"color": "gray"}), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'
        
        def determine_column_type(col):
            try:
                converted = pd.to_numeric(df[col], errors='coerce')
                if converted.isna().mean() < 0.5:
                    return "Numérique"
                else:
                    return "Qualitatif"
            except:
                return "Qualitatif"

        column_types = [determine_column_type(col) for col in df.columns]

        summary_data = {
            "Colonnes": df.columns,
            "Type": column_types,
            "Valeurs uniques": df.nunique(),
            "Valeurs manquantes": df.isna().sum(),
            "Min": [pd.NA] * len(df.columns),
            "Max": [pd.NA] * len(df.columns),
            "Moyenne": [pd.NA] * len(df.columns),
            "Médiane": [pd.NA] * len(df.columns),
            "Écart-type": [pd.NA] * len(df.columns),
            "Variance": [pd.NA] * len(df.columns)
        }

        for idx, (col, col_type) in enumerate(zip(df.columns, column_types)):
            if col_type == "Numérique":
                summary_data["Min"][idx] = round(df[col].min(), 2)
                summary_data["Max"][idx] = round(df[col].max(), 2)
                summary_data["Moyenne"][idx] = round(df[col].mean(), 2)
                summary_data["Médiane"][idx] = round(df[col].median(), 2)
                summary_data["Écart-type"][idx] = round(df[col].std(), 2)
                summary_data["Variance"][idx] = round(df[col].var(), 2)

        summary = pd.DataFrame(summary_data)

        table = dbc.Table.from_dataframe(summary, striped=True, bordered=True, hover=True, style={"color": "white"})
        return html.Div([
            html.H4("Résumé des Données", style={"color": "lightblue"}),
            html.P(f"Nombre de lignes : {df.shape[0]} | Nombre de colonnes : {df.shape[1]}", style={"color": "lightgray"}),
            html.Div(id='search-container', style={"display": "block"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='predict-container', style={"display": "none"}),
            html.Div(id='faq-container', style={"display": "none"}),
            html.Div(id='summary-table', children=table)
        ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", temp_data, 'imputation'

    return html.Div([
        html.H5(f"Affichage de {min(num_rows, len(df))} lignes sur {len(df)}.", style={"color": "#FBE4D8"}),
        render_table(df, min(num_rows, len(df))),
        html.Div(id='search-container', style={"display": "none"}),
        html.Div(id='impute-container', style={"display": "none"}),
        html.Div(id='dist-container', style={"display": "none"}),
        html.Div(id='stats-container', style={"display": "none"}),
        html.Div(id='corr-container', style={"display": "none"}),
        html.Div(id='regression-container', style={"display": "none"}),
        html.Div(id='tests-container', style={"display": "none"}),
        html.Div(id='predict-container', style={"display": "none"}),
        html.Div(id='faq-container', style={"display": "none"}),
        html.Div(id='summary-table')
    ]), f"Fichier chargé: {filename}" if filename else "Fichier chargé: Aucun", df_temp.to_dict() if not df_temp.empty else None, 'imputation'
# Callback pour mettre à jour les options de méthode d'imputation
# Callback pour mettre à jour les options de la méthode d'imputation
@app.callback(
    [Output('impute-method', 'options'),
     Output('impute-message', 'children')],
    [Input('impute-column', 'value')]
)
def update_impute_method_options(selected_column):
    global df
    if not selected_column or df.empty:
        return [], "Veuillez sélectionner une colonne."

    try:
        converted = pd.to_numeric(df[selected_column], errors='coerce')
        if converted.isna().mean() < 0.5:
            is_numeric = True
        else:
            is_numeric = False
    except:
        is_numeric = False

    if is_numeric:
        options = [
            {'label': 'Moyenne', 'value': 'mean'},
            {'label': 'Médiane', 'value': 'median'},
            {'label': 'Min', 'value': 'min'},
            {'label': 'Max', 'value': 'max'}
        ]
    else:
        options = [{'label': 'Mode', 'value': 'mode'}]
    
    return options, ""

# Callback pour appliquer l'imputation
@app.callback(
    [Output('impute-message', 'children', allow_duplicate=True),
     Output('show-results-btn', 'disabled'),
     Output('validate-data-btn', 'disabled'),
     Output('download-data-btn', 'disabled'),
     Output('temp-data-store', 'data', allow_duplicate=True)],
    [Input('apply-impute-btn', 'n_clicks')],
    [State('impute-column', 'value'),
     State('impute-method', 'value'),
     State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def apply_imputation(n_clicks, selected_column, selected_method, temp_data):
    global df
    if not n_clicks or not selected_column or not selected_method or df.empty:
        return "Veuillez sélectionner une colonne et une méthode.", True, True, True, temp_data

    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else df.copy()

    if df_temp.empty:
        return "Erreur : Aucune donnée disponible pour l'imputation.", True, True, True, temp_data

    missing_count = df_temp[selected_column].isna().sum()
    if missing_count == 0:
        return (
            f"Aucune valeur manquante trouvée dans la colonne {selected_column}.",
            False,
            False,
            False,
            df_temp.to_dict()
        )

    try:
        if selected_method == 'mean':
            df_temp[selected_column] = df_temp[selected_column].fillna(df_temp[selected_column].mean())
        elif selected_method == 'median':
            df_temp[selected_column] = df_temp[selected_column].fillna(df_temp[selected_column].median())
        elif selected_method == 'min':
            df_temp[selected_column] = df_temp[selected_column].fillna(df_temp[selected_column].min())
        elif selected_method == 'max':
            df_temp[selected_column] = df_temp[selected_column].fillna(df_temp[selected_column].max())
        elif selected_method == 'mode':
            mode_series = df_temp[selected_column].mode()
            if mode_series.empty:
                return "Erreur : Aucune valeur non manquante pour calculer le mode.", True, True, True, temp_data
            df_temp[selected_column] = df_temp[selected_column].fillna(mode_series[0])

        return (
            f"Imputation appliquée avec succès à la colonne {selected_column}. {missing_count} valeur(s) manquante(s) ont été remplacée(s).",
            False,
            False,
            False,
            df_temp.to_dict()
        )
    except Exception as e:
        return f"Erreur lors de l'imputation : {str(e)}", True, True, True, temp_data

# Callback pour afficher les résultats après imputation
@app.callback(
    [Output('impute-container', 'children'),
     Output('display-state', 'data', allow_duplicate=True)],
    [Input('show-results-btn', 'n_clicks')],
    [State('num-rows', 'value'),
     State('temp-data-store', 'data'),
     State('display-state', 'data')],
    prevent_initial_call=True
)
def show_imputation_results(show_clicks, num_rows, temp_data, display_state):
    if not show_clicks:
        return dash.no_update, dash.no_update

    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    column_options = [{'label': col, 'value': col} for col in df_temp.columns] if not df_temp.empty else []

    if df_temp.empty:
        return [
            html.P("Aucune donnée disponible après imputation.", style={"color": "gray"}),
            html.Button("Retour", id="back-btn", n_clicks=0, style={
                "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
                "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
            })
        ], 'results'
    
    return [
        html.H5("Résultats après imputation", style={"color": "lightblue"}),
        render_table(df_temp, min(num_rows, len(df_temp))),
        html.Button("Retour", id="back-btn", n_clicks=0, style={
            "margin": "10px", "fontWeight": "bold", "backgroundColor": "Darkgray",
            "color": "Black", "border": "2px solid Purple", "borderRadius": "8px"
        })
    ], 'results'
# Callback pour revenir à l'interface d'imputation
@app.callback(
    [Output('impute-container', 'children', allow_duplicate=True),
     Output('display-state', 'data', allow_duplicate=True)],
    [Input('back-btn', 'n_clicks')],
    [State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def return_to_imputation(back_clicks, temp_data):
    if not back_clicks:
        return dash.no_update, dash.no_update

    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    column_options = [{'label': col, 'value': col} for col in df_temp.columns] if not df_temp.empty else []

    return render_imputation_interface(column_options), 'imputation'

# Callback pour valider les données
@app.callback(
    [Output('output-content', 'children', allow_duplicate=True),
     Output('impute-message', 'children', allow_duplicate=True)],
    [Input('validate-data-btn', 'n_clicks')],
    [State('num-rows', 'value'),
     State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def validate_data(n_clicks, num_rows, temp_data):
    if not n_clicks:
        return dash.no_update, dash.no_update
    
    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    
    if df_temp.empty:
        return (
            html.P("Erreur : Aucune donnée disponible pour validation.", style={"color": "red"}),
            "Erreur : Aucune donnée disponible pour validation."
        )
    
    output = html.Div([
        html.H5(f"Données finales après validation - Affichage de {min(num_rows, len(df_temp))} lignes sur {len(df_temp)}.", style={"color": "white"}),
        render_table(df_temp, min(num_rows, len(df_temp))),
        html.Div(id='search-container', style={"display": "none"}),
        html.Div(id='impute-container', style={"display": "none"}),
        html.Div(id='dist-container', style={"display": "none"}),
        html.Div(id='stats-container', style={"display": "none"}),
        html.Div(id='corr-container', style={"display": "none"}),
        html.Div(id='regression-container', style={"display": "none"}),
        html.Div(id='tests-container', style={"display": "none"}),
        html.Div(id='summary-table')
    ])
    
    return (
        output,
        f"Validation réussie. Données prêtes à être téléchargées."
    )

# Callback pour télécharger les données
@app.callback(
    [Output('download-data', 'data'),
     Output('download-message', 'children')],
    [Input('download-data-btn', 'n_clicks')],
    [State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def download_data(n_clicks, temp_data):
    if not n_clicks:
        return dash.no_update, dash.no_update
    
    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    
    if df_temp.empty:
        return dash.no_update, "Erreur : Aucune donnée à télécharger."
    
    try:
        csv_string = df_temp.to_csv(index=False, encoding='utf-8')
        return (
            dict(content=csv_string, filename="donnees_imputees.csv", type="text/csv"),
            "Téléchargement initié."
        )
    except Exception as e:
        return dash.no_update, f"Erreur lors du téléchargement : {str(e)}"

# Callback pour gérer les modifications des champs éditables
@app.callback(
    [Output('output-content', 'children', allow_duplicate=True),
     Output('temp-data-store', 'data', allow_duplicate=True)],
    [Input({'type': 'editable-cell', 'index': dash.dependencies.ALL, 'column': dash.dependencies.ALL}, 'value')],
    [State('num-rows', 'value'),
     State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def update_edited_values(editable_values, num_rows, temp_data):
    global df
    ctx = dash.callback_context

    if not ctx.triggered or df.empty:
        return dash.no_update, temp_data

    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else df.copy()

    for i, prop_id in enumerate(ctx.inputs_list[0]):
        input_id = prop_id['id']
        row_index = input_id['index']
        column = input_id['column']
        value = editable_values[i]
        if row_index < len(df) and column in df.columns:
            try:
                # Convertir la valeur en numérique si la colonne est numérique
                if pd.to_numeric(df[column], errors='coerce').notna().mean() > 0.5:
                    value = pd.to_numeric(value, errors='coerce')
                df.at[row_index, column] = value
                df_temp.at[row_index, column] = value
            except Exception as e:
                continue  # Ignorer les erreurs de conversion

    return (
        html.Div([
            html.H5(f"Affichage de {min(num_rows, len(df))} lignes sur {len(df)}.", style={"color": "white"}),
            render_table(df, min(num_rows, len(df))),
            html.Div(id='search-container', style={"display": "none"}),
            html.Div(id='impute-container', style={"display": "none"}),
            html.Div(id='dist-container', style={"display": "none"}),
            html.Div(id='stats-container', style={"display": "none"}),
            html.Div(id='corr-container', style={"display": "none"}),
            html.Div(id='regression-container', style={"display": "none"}),
            html.Div(id='tests-container', style={"display": "none"}),
            html.Div(id='summary-table')
        ]),
        df_temp.to_dict()
    )
# Callback pour la recherche de colonnes
@app.callback(
    [Output('summary-table', 'children', allow_duplicate=True)],
    [Input('search-input', 'value')],
    [State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def search_columns(search_value, temp_data):
    if not search_value or not temp_data:
        return dash.no_update

    df_temp = pd.DataFrame.from_dict(temp_data)
    if df_temp.empty:
        return html.P("Aucune donnée disponible.", style={"color": "gray"})

    def determine_column_type(col):
        try:
            converted = pd.to_numeric(df_temp[col], errors='coerce')
            if converted.isna().mean() < 0.5:
                return "Numérique"
            else:
                return "Qualitatif"
        except:
            return "Qualitatif"

    column_types = [determine_column_type(col) for col in df_temp.columns]

    summary_data = {
        "Colonnes": df_temp.columns,
        "Type": column_types,
        "Valeurs uniques": df_temp.nunique(),
        "Valeurs manquantes": df_temp.isna().sum(),
        "Min": [pd.NA] * len(df_temp.columns),
        "Max": [pd.NA] * len(df_temp.columns),
        "Moyenne": [pd.NA] * len(df_temp.columns),
        "Médiane": [pd.NA] * len(df_temp.columns),
        "Écart-type": [pd.NA] * len(df_temp.columns),
        "Variance": [pd.NA] * len(df_temp.columns)
    }

    for idx, (col, col_type) in enumerate(zip(df_temp.columns, column_types)):
        if col_type == "Numérique":
            summary_data["Min"][idx] = df_temp[col].min()
            summary_data["Max"][idx] = df_temp[col].max()
            summary_data["Moyenne"][idx] = df_temp[col].mean()
            summary_data["Médiane"][idx] = df_temp[col].median()
            summary_data["Écart-type"][idx] = df_temp[col].std()
            summary_data["Variance"][idx] = df_temp[col].var()

    summary = pd.DataFrame(summary_data)

    if search_value:
        summary = summary[summary['Colonnes'].str.contains(search_value, case=False, na=False)]

    if summary.empty:
        return html.P("Aucune colonne correspondante trouvée.", style={"color": "gray"})

    table = dbc.Table.from_dataframe(summary, striped=True, bordered=True, hover=True, style={"color": "white"})
    return html.Div([
        html.H4("Résumé des Données", style={"color": "lightblue"}),
        html.P(f"Nombre de lignes : {df_temp.shape[0]} | Nombre de colonnes : {df_temp.shape[1]}", style={"color": "lightgray"}),
        table
    ])

# Callback pour la visualisation des distributions
@app.callback(
    Output('dist-graph', 'figure'),
    [Input('dist-column', 'value'),
     Input('dist-type', 'value')],
    [State('temp-data-store', 'data')]
)
def update_distribution_graph(dist_column, dist_type, temp_data):
    if not dist_column or not temp_data:
        return go.Figure()

    df_temp = pd.DataFrame.from_dict(temp_data)
    if df_temp.empty or dist_column not in df_temp.columns:
        return go.Figure()

    data = pd.to_numeric(df_temp[dist_column], errors='coerce').dropna()
    if len(data) == 0:
        return go.Figure()

    if dist_type == 'density':
        fig = go.Figure()
        hist_data = np.histogram(data, bins=30, density=True)
        hist_x = hist_data[1][:-1] + (hist_data[1][1] - hist_data[1][0]) / 2
        hist_y = hist_data[0]

        fig.add_trace(go.Scatter(
            x=hist_x,
            y=hist_y,
            mode='lines',
            fill='tozeroy',
            line=dict(color='#FFD700')
        ))

        kde = stats.gaussian_kde(data)
        x_range = np.linspace(min(data), max(data), 200)
        fig.add_trace(go.Scatter(
            x=x_range,
            y=kde(x_range),
            mode='lines',
            line=dict(color='purple')
        ))

        fig.update_layout(
            title=f"Densité de {dist_column}",
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font=dict(color="white"),
            xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
        )
        return fig

    elif dist_type == 'qqplot':
        theoretical_quantiles, sample_quantiles = stats.probplot(data, dist="norm", fit=False)[:2]
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=theoretical_quantiles,
            y=sample_quantiles,
            mode='markers',
            marker=dict(color='#FFD700', opacity=0.7)
        ))

        min_val = min(min(theoretical_quantiles), min(sample_quantiles))
        max_val = max(max(theoretical_quantiles), max(sample_quantiles))
        fig.add_trace(go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            line=dict(color='purple', dash='dash')
        ))

        fig.update_layout(
            title=f"Q-Q Plot de {dist_column}",
            xaxis_title="Quantiles théoriques",
            yaxis_title="Quantiles observés",
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font=dict(color="white"),
            xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
        )
        return fig

    return go.Figure()

# Callback pour les visualisations statistiques
@app.callback(
    [Output('hist-graph', 'figure'),
     Output('box-graph', 'figure'),
     Output('group-hist-graph', 'figure'),
     Output('group-box-graph', 'figure'),
     Output('pie-graph', 'figure'),
     Output('stacked-graph', 'figure')],
    [Input('hist-column', 'value'),
     Input('hist-bins', 'value'),
     Input('box-column', 'value'),
     Input('box-width', 'value'),
     Input('group-hist-column', 'value'),
     Input('group-hist-value-column', 'value'),
     Input('group-box-column', 'value'),
     Input('group-box-value-column', 'value'),
     Input('pie-column', 'value'),
     Input('stacked-x-column', 'value'),
     Input('stacked-y-column', 'value'),
     Input('stacked-color-column', 'value')],
    [State('temp-data-store', 'data')]
)
def update_statistics_graphs(hist_column, hist_bins, box_column, box_width, group_hist_column, group_hist_value_column, group_box_column, group_box_value_column, pie_column, stacked_x_column, stacked_y_column, stacked_color_column, temp_data):
    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()

    # Histogramme
    hist_fig = go.Figure()
    if hist_column and not df_temp.empty and hist_column in df_temp.columns:
        data = pd.to_numeric(df_temp[hist_column], errors='coerce').dropna()
        if len(data) > 0:
            hist_fig = px.histogram(
                data,
                nbins=hist_bins,
                title=f"Histogramme de {hist_column}",
                color_discrete_sequence=['#FFD700']
            )
            hist_fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    # Diagramme en boîte
    box_fig = go.Figure()
    if box_column and not df_temp.empty and box_column in df_temp.columns:
        data = pd.to_numeric(df_temp[box_column], errors='coerce').dropna()
        if len(data) > 0:
            box_fig = go.Figure()
            box_fig.add_trace(go.Box(
                y=data,
                name=box_column,
                marker_color='#FFD700',
                boxmean=True,
                width=box_width
            ))
            box_fig.update_layout(
                title=f"Diagramme en boîte de {box_column}",
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    # Histogramme groupé
    group_hist_fig = go.Figure()
    if group_hist_column and group_hist_value_column and not df_temp.empty and group_hist_column in df_temp.columns and group_hist_value_column in df_temp.columns:
        data = df_temp[[group_hist_column, group_hist_value_column]].dropna()
        data[group_hist_value_column] = pd.to_numeric(data[group_hist_value_column], errors='coerce')
        data = data.dropna()
        if len(data) > 0:
            group_hist_fig = px.histogram(
                data,
                x=group_hist_value_column,
                color=group_hist_column,
                barmode='group',
                title=f"Histogramme groupé de {group_hist_value_column} par {group_hist_column}"
            )
            group_hist_fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    # Diagramme en boîte groupé
    group_box_fig = go.Figure()
    if group_box_column and group_box_value_column and not df_temp.empty and group_box_column in df_temp.columns and group_box_value_column in df_temp.columns:
        data = df_temp[[group_box_column, group_box_value_column]].dropna()
        data[group_box_value_column] = pd.to_numeric(data[group_box_value_column], errors='coerce')
        data = data.dropna()
        if len(data) > 0:
            group_box_fig = px.box(
                data,
                x=group_box_column,
                y=group_box_value_column,
                title=f"Diagramme en boîte groupé de {group_box_value_column} par {group_box_column}",
                color=group_box_column
            )
            group_box_fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    # Diagramme à secteurs
    pie_fig = go.Figure()
    if pie_column and not df_temp.empty and pie_column in df_temp.columns:
        data = df_temp[pie_column].dropna()
        if len(data) > 0:
            pie_fig = px.pie(
                names=data,
                title=f"Diagramme à secteurs de {pie_column}"
            )
            pie_fig.update_traces(textinfo='percent+label')
            pie_fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    # Graphique croisé
    stacked_fig = go.Figure()
    if stacked_x_column and stacked_y_column and stacked_color_column and not df_temp.empty and all(col in df_temp.columns for col in [stacked_x_column, stacked_y_column, stacked_color_column]):
        data = df_temp[[stacked_x_column, stacked_y_column, stacked_color_column]].dropna()
        if len(data) > 0:
            pivot_table = data.pivot_table(index=stacked_x_column, columns=stacked_color_column, values=stacked_y_column, aggfunc='count', fill_value=0)
            for col in pivot_table.columns:
                stacked_fig.add_trace(go.Bar(
                    x=pivot_table.index,
                    y=pivot_table[col],
                    name=str(col)
                ))
            stacked_fig.update_layout(
                barmode='stack',
                title=f"Graphique croisé : {stacked_y_column} par {stacked_x_column} (coloré par {stacked_color_column})",
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="white"),
                xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
                legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
            )

    return hist_fig, box_fig, group_hist_fig, group_box_fig, pie_fig, stacked_fig

# Callback pour la visualisation des corrélations
@app.callback(
    Output('corr-graph', 'figure'),
    [Input('corr-method', 'value')],
    [State('temp-data-store', 'data')]
)
def update_correlation_graph(corr_method, temp_data):
    if not temp_data:
        return go.Figure()

    df_temp = pd.DataFrame.from_dict(temp_data)
    if df_temp.empty:
        return go.Figure()

    numeric_columns = [col for col in df_temp.columns if pd.to_numeric(df_temp[col], errors='coerce').notna().mean() > 0.5]
    if not numeric_columns:
        return go.Figure()

    df_numeric = df_temp[numeric_columns].apply(pd.to_numeric, errors='coerce')
    corr_matrix = df_numeric.corr()

    if corr_method == 'number':
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='Viridis',
            text=corr_matrix.values,
            texttemplate="%{text:.2f}",
            textfont={"size": 12},
            showscale=True
        ))
    else:
        fig = go.Figure()
        for i, row in enumerate(corr_matrix.values):
            for j, value in enumerate(row):
                if i != j:
                    fig.add_trace(go.Scatter(
                        x=[j],
                        y=[i],
                        mode='markers',
                        marker=dict(
                            size=abs(value) * 30,
                            color=value,
                            colorscale='Viridis',
                            showscale=True
                        ),
                        text=f"{corr_matrix.index[i]} vs {corr_matrix.columns[j]}: {value:.2f}",
                        hoverinfo='text'
                    ))

        fig.update_layout(
            xaxis=dict(
                tickmode='array',
                tickvals=list(range(len(corr_matrix.columns))),
                ticktext=corr_matrix.columns,
                showgrid=False,
                zeroline=False
            ),
            yaxis=dict(
                tickmode='array',
                tickvals=list(range(len(corr_matrix.index))),
                ticktext=corr_matrix.index,
                showgrid=False,
                zeroline=False,
                autorange="reversed"
            ),
            title="Corrélation (Cercles)",
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font=dict(color="white"),
            legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
        )

    return fig

# Callback pour exécuter la régression linéaire
# Callback pour exécuter la régression linéaire
@app.callback(
    [Output('regression-results', 'children'),
     Output('download-regression-btn', 'disabled'),
     Output('regression-message', 'children'),
     Output('regression-results-store', 'data')],
    [Input('run-regression-btn', 'n_clicks')],
    [State('regression-x-column', 'value'),
     State('regression-y-column', 'value'),
     State('temp-data-store', 'data')],
    prevent_initial_call=True
)
def run_regression(n_clicks, x_column, y_column, temp_data):
    if not n_clicks or not x_column or not y_column or not temp_data:
        return "", True, "Veuillez sélectionner les colonnes X et Y.", None

    df_temp = pd.DataFrame.from_dict(temp_data)
    if df_temp.empty or x_column not in df_temp.columns or y_column not in df_temp.columns:
        return "", True, "Erreur : Données ou colonnes manquantes.", None

    try:
        X = pd.to_numeric(df_temp[x_column], errors='coerce').dropna()
        y = pd.to_numeric(df_temp[y_column], errors='coerce').dropna()
        data = pd.DataFrame({x_column: X, y_column: y}).dropna()
        if data.empty:
            return "", True, "Erreur : Aucune donnée valide pour la régression.", None

        X_data = data[x_column].values.reshape(-1, 1)
        y_data = data[y_column].values
        reg = LinearRegression()
        reg.fit(X_data, y_data)
        y_pred = reg.predict(X_data)
        mse = mean_squared_error(y_data, y_pred)

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=data[x_column],
            y=data[y_column],
            mode='markers',
            name='Données',
            marker=dict(color='#FFD700')
        ))
        fig.add_trace(go.Scatter(
            x=data[x_column],
            y=y_pred,
            mode='lines',
            name='Régression',
            line=dict(color='purple')
        ))
        fig.update_layout(
            title=f"Régression Linéaire : {y_column} vs {x_column}",
            xaxis_title=x_column,
            yaxis_title=y_column,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            font=dict(color="white"),
            xaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            yaxis=dict(gridcolor='rgba(255,255,255,0.1)', zerolinecolor='rgba(255,255,255,0.2)'),
            legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple', borderwidth=1)
        )

        results = {
            'slope': reg.coef_[0],
            'intercept': reg.intercept_,
            'mse': mse,
            'x_column': x_column,
            'y_column': y_column
        }

        summary = html.Div([
            html.P(f"Pente : {reg.coef_[0]:.2f}", style={"color": "lightgray"}),
            html.P(f"Ordonnée à l'origine : {reg.intercept_:.2f}", style={"color": "lightgray"}),
            html.P(f"Erreur quadratique moyenne (MSE) : {mse:.2f}", style={"color": "lightgray"}),
            dcc.Graph(figure=fig)
        ])

        return summary, False, "Régression linéaire exécutée avec succès.", results
    except Exception as e:
        return "", True, f"Erreur lors de la régression : {str(e)}", None

# Callback pour télécharger les résultats de la régression
@app.callback(
    Output('download-regression', 'data'),
    [Input('download-regression-btn', 'n_clicks')],
    [State('regression-results-store', 'data')],
    prevent_initial_call=True
)
def download_regression_results(n_clicks, regression_results):
    if not n_clicks or not regression_results:
        return dash.no_update

    result_df = pd.DataFrame({
        'Métrique': ['Pente', 'Ordonnée à l\'origine', 'MSE'],
        'Valeur': [regression_results['slope'], regression_results['intercept'], regression_results['mse']],
        'Variable X': [regression_results['x_column']] * 3,
        'Variable Y': [regression_results['y_column']] * 3
    })

    csv_string = result_df.to_csv(index=False, encoding='utf-8')
    return dict(content=csv_string, filename="resultats_regression.csv", type="text/csv")

@app.callback(
    Output('test-options-container', 'children'),
    [Input('test-type', 'value')],
    [State('temp-data-store', 'data')]
)
def update_test_options(test_type, temp_data):
    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    if not test_type or df_temp.empty:
        return []

    all_columns = list(df_temp.columns)
    # Colonnes numériques : au moins 50% des valeurs peuvent être converties en nombres
    numeric_columns = [col for col in df_temp.columns if pd.to_numeric(df_temp[col], errors='coerce').notna().mean() > 0.5]
    # Colonnes catégoriques : colonnes qui ne sont pas majoritairement numériques
    categorical_columns = [col for col in df_temp.columns if col not in numeric_columns]

    components = []

    if test_type == 't-test':
        components.extend([
            html.Label("Sélectionner la colonne de regroupement (catégorique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='group-column',
                options=[{'label': col, 'value': col} for col in categorical_columns],
                placeholder="Sélectionnez une colonne catégorique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            ),
            html.Label("Sélectionner la colonne des valeurs (numérique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='value-column',
                options=[{'label': col, 'value': col} for col in numeric_columns],
                placeholder="Sélectionnez une colonne numérique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            )
        ])
    elif test_type == 'normality':
        components.extend([
            html.Label("Sélectionner la colonne pour le test de normalité (numérique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='test-column',
                options=[{'label': col, 'value': col} for col in numeric_columns],
                placeholder="Sélectionnez une colonne numérique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            )
        ])
    elif test_type == 'chi-squared':
        components.extend([
            html.Label("Sélectionner la première colonne (catégorique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='chi2-column1',
                options=[{'label': col, 'value': col} for col in categorical_columns],
                placeholder="Sélectionnez une colonne catégorique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            ),
            html.Label("Sélectionner la deuxième colonne (catégorique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='chi2-column2',
                options=[{'label': col, 'value': col} for col in categorical_columns],
                placeholder="Sélectionnez une colonne catégorique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            )
        ])
    elif test_type == 'anova':
        components.extend([
            html.Label("Sélectionner la colonne de regroupement (catégorique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='group-column',
                options=[{'label': col, 'value': col} for col in categorical_columns],
                placeholder="Sélectionnez une colonne catégorique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            ),
            html.Label("Sélectionner la colonne des valeurs (numérique)", style={"color": "lightgray"}),
            dcc.Dropdown(
                id='value-column',
                options=[{'label': col, 'value': col} for col in numeric_columns],
                placeholder="Sélectionnez une colonne numérique",
                style={
                    "width": "50%", 
                    "marginBottom": "10px", 
                    "backgroundColor": "white", 
                    "color": "black"
                }
            )
        ])

    # Ajouter des placeholders cachés pour les composants non utilisés
    if test_type != 'normality':
        components.append(
            dcc.Dropdown(id='test-column', options=[], value=None, style={'display': 'none'})
        )
    if test_type not in ['t-test', 'anova']:
        components.extend([
            dcc.Dropdown(id='group-column', options=[], value=None, style={'display': 'none'}),
            dcc.Dropdown(id='value-column', options=[], value=None, style={'display': 'none'})
        ])
    if test_type != 'chi-squared':
        components.extend([
            dcc.Dropdown(id='chi2-column1', options=[], value=None, style={'display': 'none'}),
            dcc.Dropdown(id='chi2-column2', options=[], value=None, style={'display': 'none'})
        ])

    return components
@app.callback(
    [Output('test-results', 'children'),
     Output('download-test-btn', 'disabled'),
     Output('test-message', 'children'),
     Output('test-results-store', 'data')],
    [Input('run-test-btn', 'n_clicks')],
    [State('test-type', 'value'),
     State('test-column', 'value'),
     State('group-column', 'value'),
     State('value-column', 'value'),
     State('chi2-column1', 'value'),
     State('chi2-column2', 'value'),
     State('temp-data-store', 'data')]
)
def run_statistical_test(n_clicks, test_type, test_column, group_column, value_column, chi2_column1, chi2_column2, temp_data):
    if not n_clicks or not test_type or not temp_data:
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update

    df_temp = pd.DataFrame.from_dict(temp_data)
    if df_temp.empty:
        return html.P("Erreur : Aucune donnée disponible.", style={"color": "red"}), True, "Erreur : Aucune donnée disponible.", None

    results = {}
    try:
        # Vérifier les colonnes numériques et catégoriques
        numeric_columns = [col for col in df_temp.columns if pd.to_numeric(df_temp[col], errors='coerce').notna().mean() > 0.5]
        categorical_columns = [col for col in df_temp.columns if col not in numeric_columns]

        if test_type == 't-test':
            if not group_column or not value_column:
                return html.P("Veuillez sélectionner une colonne de regroupement et une colonne de valeurs numériques.", style={"color": "red"}), True, "Veuillez sélectionner les colonnes nécessaires.", None
            if value_column not in numeric_columns:
                return html.P("Erreur : La colonne des valeurs doit contenir des données numériques.", style={"color": "red"}), True, "La colonne des valeurs doit être numérique.", None
            groups = df_temp.groupby(group_column)[value_column].apply(lambda x: x.dropna().tolist())
            if len(groups) != 2:
                return html.P("Erreur : Le t-test nécessite exactement 2 groupes.", style={"color": "red"}), True, "Le t-test nécessite 2 groupes.", None
            if any(len(g) < 2 for g in groups):
                return html.P("Erreur : Chaque groupe doit contenir au moins 2 observations.", style={"color": "red"}), True, "Chaque groupe doit avoir au moins 2 observations.", None
            stat, p_value = ttest_ind(groups.iloc[0], groups.iloc[1])
            results = {
                "Test": "t-test",
                "Statistique": stat,
                "p-valeur": p_value,
                "Interprétation": "Différence significative entre les groupes" if p_value < 0.05 else "Pas de différence significative"
            }

        elif test_type == 'normality':
            if not test_column:
                return html.P("Veuillez sélectionner une colonne numérique.", style={"color": "red"}), True, "Veuillez sélectionner une colonne numérique.", None
            if test_column not in numeric_columns:
                return html.P("Erreur : La colonne sélectionnée doit contenir des données numériques.", style={"color": "red"}), True, "La colonne doit être numérique.", None
            data = df_temp[test_column].dropna()
            if len(data) < 3:
                return html.P("Erreur : Pas assez de données pour effectuer le test de normalité (au moins 3 valeurs nécessaires).", style={"color": "red"}), True, "Pas assez de données.", None
            stat, p_value = shapiro(data)
            results = {
                "Test": "Test de normalité (Shapiro-Wilk)",
                "Statistique": stat,
                "p-valeur": p_value,
                "Interprétation": "Données non normales" if p_value < 0.05 else "Données normales"
            }

        elif test_type == 'chi-squared':
            if not chi2_column1 or not chi2_column2:
                return html.P("Veuillez sélectionner deux colonnes pour le test Chi-squared.", style={"color": "red"}), True, "Veuillez sélectionner deux colonnes.", None
            contingency_table = pd.crosstab(df_temp[chi2_column1], df_temp[chi2_column2])
            stat, p_value, dof, _ = chi2_contingency(contingency_table)
            results = {
                "Test": "Chi-squared",
                "Statistique": stat,
                "p-valeur": p_value,
                "Degré de liberté": dof,
                "Interprétation": "Dépendance significative entre les variables" if p_value < 0.05 else "Pas de dépendance significative"
            }

        elif test_type == 'anova':
            if not group_column or not value_column:
                return html.P("Veuillez sélectionner une colonne de regroupement et une colonne de valeurs numériques.", style={"color": "red"}), True, "Veuillez sélectionner les colonnes nécessaires.", None
            if value_column not in numeric_columns:
                return html.P("Erreur : La colonne des valeurs doit contenir des données numériques.", style={"color": "red"}), True, "La colonne des valeurs doit être numérique.", None
            groups = df_temp.groupby(group_column)[value_column].apply(lambda x: x.dropna().tolist())
            if any(len(g) < 2 for g in groups):
                return html.P("Erreur : Chaque groupe doit contenir au moins 2 observations.", style={"color": "red"}), True, "Chaque groupe doit avoir au moins 2 observations.", None
            stat, p_value = f_oneway(*groups)
            results = {
                "Test": "ANOVA",
                "Statistique": stat,
                "p-valeur": p_value,
                "Interprétation": "Différence significative entre les groupes" if p_value < 0.05 else "Pas de différence significative"
            }

        # Afficher les résultats sous forme de tableau
        result_df = pd.DataFrame([results])
        result_table = dbc.Table.from_dataframe(result_df, striped=True, bordered=True, hover=True, style={"color": "white"})
        return result_table, False, "Test exécuté avec succès.", results

    except Exception as e:
        return html.P(f"Erreur lors de l'exécution du test : {str(e)}", style={"color": "red"}), True, f"Erreur : {str(e)}", None
@app.callback(
    [Output('download-test', 'data'),
     Output('test-message', 'children', allow_duplicate=True)],
    [Input('download-test-btn', 'n_clicks')],
    [State('test-results-store', 'data')],
    prevent_initial_call=True
)
def download_test_results(n_clicks, test_results):
    if not n_clicks or not test_results:
        return dash.no_update, dash.no_update

    result_df = pd.DataFrame([test_results])
    try:
        csv_string = result_df.to_csv(index=False, encoding='utf-8')
        return (
            dict(content=csv_string, filename="resultats_tests.csv", type="text/csv"),
            "Téléchargement initié."
        )
    except Exception as e:
        return dash.no_update, f"Erreur lors du téléchargement : {str(e)}"
@app.callback(
    [Output('t-test-graph', 'figure'),
     Output('t-test-graph', 'style'),
     Output('normality-graph', 'figure'),
     Output('normality-graph', 'style'),
     Output('chi-squared-graph', 'figure'),
     Output('chi-squared-graph', 'style'),
     Output('anova-graph', 'figure'),
     Output('anova-graph', 'style')],
    [Input('test-results-store', 'data')],
    [State('test-type', 'value'),
     State('temp-data-store', 'data'),
     State('test-column', 'value'),
     State('group-column', 'value'),
     State('value-column', 'value'),
     State('chi2-column1', 'value'),
     State('chi2-column2', 'value')]
)
def update_test_graphs(test_results, test_type, temp_data, test_col, group_col, value_col, chi2_col1, chi2_col2):
    df_temp = pd.DataFrame.from_dict(temp_data) if temp_data else pd.DataFrame()
    t_test_fig = go.Figure()
    normality_fig = go.Figure()
    chi_squared_fig = go.Figure()
    anova_fig = go.Figure()

    t_test_style = {'display': 'none'}
    normality_style = {'display': 'none'}
    chi_squared_style = {'display': 'none'}
    anova_style = {'display': 'none'}

    if test_results and df_temp is not None and not df_temp.empty:
        if test_type == 't-test' and 'Statistique' in test_results and group_col and value_col:
            groups = df_temp.groupby(group_col)[value_col].apply(lambda x: x.dropna().tolist())
            if len(groups) == 2:
                t_test_fig = go.Figure()
                t_test_fig.add_trace(go.Box(
                    y=groups.iloc[0],
                    name='Groupe 1',
                    marker_color='#FFD700',
                    boxpoints='all',  # Show all points
                    jitter=0.3,  # Add some jitter for visibility
                    pointpos=-1.8  # Position points to the left
                ))
                t_test_fig.add_trace(go.Box(
                    y=groups.iloc[1],
                    name='Groupe 2',
                    marker_color='#FFD700',
                    boxpoints='all',
                    jitter=0.3,
                    pointpos=-1.8
                ))
                t_test_fig.update_layout(
                    title="t-test : Distribution des groupes",
                    xaxis_title="Groupes",
                    yaxis_title="Valeurs",
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(color="white"),
                    xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    yaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple')
                )
                t_test_style = {'display': 'block'}

        elif test_type == 'normality' and 'Statistique' in test_results and test_col:
            if test_col in df_temp.columns:
                data = pd.to_numeric(df_temp[test_col], errors='coerce').dropna()
                normality_fig = px.histogram(data, nbins=30, title="Distribution pour le test de normalité")
                normality_fig.add_vline(x=np.mean(data), line_dash="dash", line_color="purple")
                normality_fig.update_layout(
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(color="white"),
                    xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    yaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple')
                )
                normality_style = {'display': 'block'}

        elif test_type == 'chi-squared' and 'Statistique' in test_results and chi2_col1 and chi2_col2:
            if chi2_col1 in df_temp.columns and chi2_col2 in df_temp.columns:
                contingency_table = pd.crosstab(df_temp[chi2_col1], df_temp[chi2_col2])
                chi_squared_fig = px.imshow(contingency_table, text_auto=True, aspect="auto", title="Tableau de contingence")
                chi_squared_fig.update_layout(
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(color="white"),
                    xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    yaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    coloraxis_showscale=False
                )
                chi_squared_style = {'display': 'block'}

        elif test_type == 'anova' and 'Statistique' in test_results and group_col and value_col:
            if group_col in df_temp.columns and value_col in df_temp.columns:
                groups = df_temp.groupby(group_col)[value_col].apply(lambda x: x.dropna().tolist())
                anova_fig = go.Figure()
                for i, (group_name, group_data) in enumerate(groups.items()):
                    anova_fig.add_trace(go.Box(
                        y=group_data,
                        name=str(group_name),
                        marker_color='#FFD700',
                        boxpoints='all',
                        jitter=0.3,
                        pointpos=-1.8
                    ))
                anova_fig.update_layout(
                    title="ANOVA : Distribution par groupe",
                    xaxis_title="Groupes",
                    yaxis_title="Valeurs",
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(color="white"),
                    xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    yaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
                    legend=dict(font=dict(color="white"), bgcolor='rgba(0,0,0,0.5)', bordercolor='purple')
                )
                anova_style = {'display': 'block'}

    return t_test_fig, t_test_style, normality_fig, normality_style, chi_squared_fig, chi_squared_style, anova_fig, anova_style
@app.callback(
    Output('prediction-results', 'children'),
    [Input('predict-btn', 'n_clicks'),
     Input('predict-manual-btn', 'n_clicks')],
    [State('upload-prediction-data', 'contents'),
     State('upload-prediction-data', 'filename'),
     State('input-age', 'value'),
     State('input-mmse', 'value'),
     State('input-cdr', 'value'),
     State('input-education', 'value'),
     State('input-family', 'value')],
    prevent_initial_call=True
)
def predict_disease(n_clicks, n_clicks_manual, contents, filename, age, mmse, cdr, education, family):
    ctx = dash.callback_context
    if not ctx.triggered:
        return "Veuillez charger un fichier ou saisir les données pour la prédiction."

    if model is None or scaler is None:
        return html.P("Erreur : Modèle ou scaler non chargé.", style={"color": "red"})

    # Si le bouton "Prédire" (fichier CSV) est cliqué
    if ctx.triggered[0]['prop_id'] == 'predict-btn.n_clicks' and contents:
        try:
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            if filename.endswith('.xlsx'):
                patient_data = pd.read_excel(io.BytesIO(decoded))
            else:
                try:
                    patient_data = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                except UnicodeDecodeError:
                    try:
                        patient_data = pd.read_csv(io.StringIO(decoded.decode('latin-1')))
                    except UnicodeDecodeError:
                        patient_data = pd.read_csv(io.StringIO(decoded.decode('cp1252')))
        except Exception as e:
            return html.P(f"Erreur lors de la lecture du fichier : {str(e)}", style={"color": "red"})
        is_manual = False
    # Si le bouton "Prédire manuellement" est cliqué
    else:
        if any(v is None for v in [age, mmse, cdr, education, family]):
            return html.P("Erreur : Veuillez remplir tous les champs.", style={"color": "red"})
        patient_data = pd.DataFrame({
            'age': [age],
            'mmse_score': [mmse],
            'cdr': [cdr],
            'education_level': [education],
            'family_history': [family]
        })
        is_manual = True

    try:
        required_columns = ['age', 'mmse_score', 'cdr', 'education_level', 'family_history']
        missing_columns = [col for col in required_columns if col not in patient_data.columns]
        if missing_columns:
            return f"Erreur : Les colonnes suivantes sont manquantes : {', '.join(missing_columns)}."

        X = patient_data[required_columns]
        X_scaled = scaler.transform(X)

        prediction = model.predict(X_scaled)
        prediction_proba = model.predict_proba(X_scaled)

        results = []
        for i, pred in enumerate(prediction):
            result = "Malade" if pred == 1 else "Sain"
            confidence = max(prediction_proba[i]) * 100
            risk_score = prediction_proba[i][1] * 100  # Probabilité d'avoir Alzheimer

            # Créer une jauge de risque
            fig = go.Figure(go.Indicator(
                mode="gauge+number",
                value=risk_score,
                title={'text': f"Risque d'Alzheimer - Patient {i+1}", 'font': {'color': 'lightblue'}},
                gauge={
                    'axis': {'range': [0, 100], 'tickfont': {'color': 'lightgray'}},
                    'bar': {'color': "purple"},
                    'steps': [
                        {'range': [0, 33], 'color': "green"},
                        {'range': [33, 66], 'color': "orange"},
                        {'range': [66, 100], 'color': "red"}
                    ],
                    'threshold': {
                        'line': {'color': "white", 'width': 4},
                        'thickness': 0.75,
                        'value': 50
                    }
                },
                number={'suffix': "%", 'font': {'color': 'lightgray'}}
            ))
            fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font=dict(color="lightgray"),
                margin=dict(l=20, r=20, t=50, b=20),
                height=200
            )

            results.append(
                html.Div([
                    html.H6(f"Patient {i+1}", style={"color": "lightblue"}),
                    html.P(f"Prédiction : {result}", style={"color": "lightgray"}),
                    html.P(f"Confiance : {confidence:.2f}%", style={"color": "lightgray"}),
                    dcc.Graph(figure=fig, style={"height": "200px"}),
                    html.Hr()
                ])
            )

        performance_metrics = []
        correct_predictions = 0
        incorrect_predictions = 0
        if 'Résultat' in patient_data.columns:
            from sklearn.metrics import accuracy_score, precision_recall_fscore_support
            true_labels = patient_data['Résultat'].values
            accuracy = accuracy_score(true_labels, prediction)
            precision, recall, f1, _ = precision_recall_fscore_support(true_labels, prediction, average='binary')
            for pred, true in zip(prediction, true_labels):
                if pred == true:
                    correct_predictions += 1
                else:
                    incorrect_predictions += 1
            performance_metrics = [
                html.H5("Performance du modèle", style={"color": "lightblue"}),
                html.P(f"Précision globale (Accuracy) : {accuracy:.2f}", style={"color": "lightgray"}),
                html.P(f"Précision (Precision) : {precision:.2f}", style={"color": "lightgray"}),
                html.P(f"Rappel (Recall) : {recall:.2f}", style={"color": "lightgray"}),
                html.P(f"F1-Score : {f1:.2f}", style={"color": "lightgray"}),
                html.P(f"Nombre de prédictions correctes : {correct_predictions}", style={"color": "lightgray"}),
                html.P(f"Nombre de prédictions incorrectes : {incorrect_predictions}", style={"color": "lightgray"}),
            ]

        return html.Div([
            html.H5("Résultat de la Prédiction Manuelle" if is_manual else "Résultat de la Prédiction", style={"color": "lightblue"}),
            *results,
            *performance_metrics
        ])
    except Exception as e:
        return html.P(f"Erreur lors de la prédiction : {str(e)}", style={"color": "red"})
# Callback pour basculer entre l'accueil et l'interface principale
@app.callback(
    [Output('welcome-container', 'style'),
     Output('main-container', 'style'),
     Output('page-state', 'data')],
    [Input('start-btn', 'n_clicks'),
     Input('back-to-welcome-btn', 'n_clicks')],
    State('page-state', 'data')
)
def toggle_page(start_clicks, back_clicks, current_state):
    ctx = dash.callback_context
    if not ctx.triggered:
        return {'display': 'block'}, {'display': 'none'}, 'welcome'

    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'start-btn' and start_clicks:
        return {'display': 'none'}, {'display': 'block'}, 'main'
    elif trigger_id == 'back-to-welcome-btn' and back_clicks:
        return {'display': 'block'}, {'display': 'none'}, 'welcome'
    
    return {'display': 'block' if current_state == 'welcome' else 'none'}, \
           {'display': 'none' if current_state == 'welcome' else 'block'}, \
           current_state
# Callback pour afficher les questions de la catégorie sélectionnée
@app.callback(
    Output('faq-content', 'children'),
    [Input('faq-tabs', 'value')]
)
def update_faq_content(tab_value):
    questions = faq_data.get(tab_value, {})
    content = [
        html.Ul([
            html.Li(html.A(question, href="#", id=f"faq-{tab_value}-{i}", style={"color": "#C71585", "cursor": "pointer"}))
            for i, question in enumerate(questions.keys())
        ], style={"listStyleType": "none", "padding": "0"})
    ]
    return content + [html.Div(id=f"faq-answer-{tab_value}", style={"marginTop": "10px"})]

for tab_value in ["demarrage", "nettoyage", "visualisation", "analyse", "prediction", "depanne"]:
    @app.callback(
        Output(f'faq-answer-{tab_value}', 'children'),
        [Input(f'faq-{tab_value}-{i}', 'n_clicks') for i in range(len(faq_data[tab_value]))],
        [State('faq-tabs', 'value')]
    )
    def display_answer(*args):
        n_clicks_list = args[:-1]  # Liste des n_clicks pour chaque question
        tab_value = args[-1]  # Dernier argument est le State('faq-tabs', 'value')
        
        ctx = dash.callback_context
        if not ctx.triggered or not any(n_clicks_list):
            return ""
        
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
        question_index = int(triggered_id.split('-')[-1])
        questions = list(faq_data.get(tab_value, {}).keys())
        if 0 <= question_index < len(questions):
            return html.Div([
                html.P("Réponse :", style={
                    'color': '#FBE4D8',
                    'fontWeight': 'bold',
                    'marginBottom': '5px'
                }),
                html.P(faq_data[tab_value][questions[question_index]], style={
                    'color': 'white',
                    'padding': '15px',
                    'backgroundColor': 'rgba(255, 255, 255, 0.1)',
                    'borderRadius': '5px',
                    'borderLeft': '4px solid #00ffcc'
                })
            ])
        return ""
# Lancer l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8063)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_26420\4206073772.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\lenovo\AppData\Local\Temp\ipykernel_26420\4206073772.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Modèle et scaler sauvegardés avec succès sous 'alzheimer_model.pkl' et 'scaler.pkl'.
Modèle et scaler chargés avec succès.


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True`

Dash app running on http://127.0.0.1:8063/
